# A Boolean Model for Information Retrieval

In this project we are going to implement a **Boolean Retrieval System**. This one is able to answer Boolean queries, Phrase queries, Wildcard queries and to perform Spelling Correction too.

In [1]:
from functools import total_ordering, reduce
import csv
import os
import re
import bisect
import pickle
import time

First of all the we need to think about the overall structure of a System of this kind:
1. In order to implement properly the **Index** we're going to use, first we have to define what is a **Posting**, a **Posting List** and a **Term**. 
2. Then we can implement some **Pre-processing steps** like **Normalization** and **Tokenization**.
3. Now it is possible to define the **Index** itself.
4. Since we decided to include also **Spelling Correction** as a feature we have to define it too.
5. Finally we will able to build our **IR System**.

## Posting and Posting List

A **Posting** is the atomic element of a Posting List. It contains the **docID** of the document in which a term appears and the **list** of its **positions** within the document. This latter attribute is included because it will become useful when dealing with Phrase queries.

In [2]:
@total_ordering
class Posting:
    """
    Posting is the atomic element of a Posting List.
    It contains the docID of the document in which the term
    is present and the list of its positions in the list.
    This latter attribute is useful when dealing with phrase
    queries (positional index).
    """
    def __init__(self, docID:int, position= None):

        self._docID = docID
        self._positions = [position]

    def getFromCorpus(self, corpus):
        """
        Given a corpus, return the document correspondent to
        the specified docID.
        """
        return corpus[self._docID]
    
    def __eq__(self, other):
        return self._docID == other._docID
    
    def __gt__(self, other): 
        return self._docID > other._docID
    
    def __repr__(self):
        return f"DocID is {self._docID} and the list of positions is {self._positions}"

    def addPosition(self, position: int):
        """
        Add position of a term in a document to its position list.
        """
        self._positions.append(position)
    
    # Needed otherwise we couldn't perform union (because sets need hashable items)
    def __hash__(self):
        return hash(repr(self))

A **Posting List**, as the name tells, is a list made of Postings related to a specific Term. A Posting List can be initialized essentially in two ways: 
1. One way is to start from a docID;
2. The other is to use another list that is made up of Postings. 

We assume that every Posting List we are considering is ordered in an ascending way (this allows to compute efficiently both intersecton ad union operations, as we saw in the theory).

In [3]:
class PostingList:
    """
    A Posting List contains a list of the docIDs in which
    a specific term appears. 
    We specify two class methods to initialize a Posting List
    starting either from a docID or from another list that 
    contains postings.
    We assume that every Posting List that we are considering
    is ordered in an ascending way.
    """
    def __init__(self):
        self._postings = []

    @classmethod
    def fromDocID(cls, docID: int, position:list = None):
        posting_list = cls()
        posting_list._postings = [Posting(docID, position)]

        return posting_list
    
    @classmethod
    def fromPostingList(cls, plist:list):
        posting_list = cls()
        posting_list._postings = plist

        return posting_list
    
    def merge(self, other):
        """
        Merge two Posting Lists.
        """
        assert len(other._postings) > 0, "Error: Posting List is empty, cannot merge!"
        i = 0
        last = self._postings[-1]
        while (i < len(other._postings) and last == other._postings[i]):
            if last._positions:
                last._positions.extend(other._postings[i]._positions)
            i += 1
        self._postings += other._postings[i:]
    
    def intersection(self, other):
        """
        Perform intersection between two Posting Lists.
        """
        intersection = [posting for posting in self._postings if posting in other._postings]

        return PostingList.fromPostingList(intersection)
    
    def union(self, other):
        """
        Perform union between two Posting Lists.
        """
        union = list(set(self._postings + other._postings))

        return PostingList.fromPostingList(union)

    def positionalSearch(self, other, k:int = 1):
        """
        Perform intersection between two Posting Lists in the 
        case of a Positional Index. We need to check if terms 
        given in the query are subsequent or not.
        With parameter k we can specify the number of words 
        that can separate the two terms present in the query.
        """
        pos_intersection = []
        self_plist_index = 0
        other_plist_index = 0
        while (self_plist_index<len(self._postings) and other_plist_index<len(other._postings)):
            if (self._postings[self_plist_index] == other._postings[other_plist_index]):
                self_position = 0
                other_position = 0
                while(self_position < len(self._postings[self_plist_index]._positions) and other_position < len(other._postings[other_plist_index]._positions)):
                    if self._postings[self_plist_index]._positions[self_position] + k == other._postings[other_plist_index]._positions[other_position]:
                        if not pos_intersection:
                            pos_intersection.append(Posting(self._postings[self_plist_index]._docID, self._postings[self_plist_index]._positions[self_position]))
                        else:
                            pos_intersection[-1].addPosition(self._postings[self_plist_index]._positions[self_position])
                        self_position += 1
                        other_position += 1
                    elif self._postings[self_plist_index]._positions[self_position] + k > other._postings[other_plist_index]._positions[other_position]:
                        other_position += 1
                    else:
                        self_position += 1
                self_plist_index += 1
                other_plist_index += 1
            elif self._postings[self_plist_index] < other._postings[other_plist_index]:
                self_plist_index += 1
            else:
                other_plist_index += 1
        
        return PostingList.fromPostingList(pos_intersection)

    def getFromCorpus(self, corpus):
        """
        Retrieve documents from docIDs present in the Posting List.
        """
        return list(map(lambda x: x.getFromCorpus(corpus), self._postings))
    
    def __getitem__(self, key):
        return self._postings[key]
        
    def __repr__(self):
        return "\n".join(map(str, self._postings))

## Term

A **Term** is made up of a word and its associated Posting List.

In [4]:
class ImpossibleMergeError(Exception):
    pass

@total_ordering
class Term:
    """
    A Term is made up of a word and its associated Posting List.
    We give the availability also to construct a term given the 
    word and a single docID.
    """
    def __init__(self, term:str = None, plist:'PostingList' = None):
        self.term = term
        self.posting_list = plist
    
    @classmethod
    def fromDocID(cls, term:str, docID:int, position:int = None):
        t = cls()
        t.term = term
        t.posting_list = PostingList.fromDocID(docID, position)

        return t
    
    @classmethod
    def fromPostingList(cls, term:str, plist:'PostingList'):
        t = cls()
        t.term = term
        t.posting_list = plist

        return t
    
    def merge(self, other):
        """
        Merge the Posting Lists of two terms.
        """
        if (self.term == other.term):
            self.posting_list.merge(other.posting_list)
        else:
            raise ImpossibleMergeError
    
    def __eq__(self, other):
        return self.term == other.term
    
    def __gt__(self, other):
        return self.term > other.term
    
    def __repr__(self):
        return self.term + ": " + repr(self.posting_list)

## Normalization and Tokenization

**Normalization** is the process of removing punctuation, accents, diacritics and capitalization in order to transform a text into a single canonical form, guaranteeing consistency for every operation performed.

**Tokenization** instead is the process of subdividing a text into tokens (a _Token_ is simply a sequence of characters). This is extremely important otherwise we wouldn't be able to build a Dictionary and so the Index.

In [5]:
def normalize(text):
    """
    Normalization function to remove punctuation and to
    lowercase the entire text.
    Basically it keeps all the words, spaces, hyphens
    and also the end of the word symbol "$" (important 
    since we are dealing with wildcard queries too).
    """
    no_punctuation = re.sub(r'[^\w^\s^-^\$]','',text)
    lowercase = no_punctuation.lower()
    return lowercase

def tokenize(post_text):
    """
    Performs tokenization on a text and
    returns a list of all of the tokens found.
    """
    text = normalize(post_text.description)
    return list(text.split())

## Index

The **Index** object we are going to implement is kind of a special one because there coexist actually three different indices:
1. An **Inverted Index**, that we remember contains every term present in the corpus and its relative Posting List;
2. a **Positional Index**, that, in addition to the previous case, stores in each Posting a list of the positions in which the term appears;
3. lastly a **Permuterm Index**, in which for every term we insert all of its possible rotations in the dictionary (this allows to answer General and Multiple Wildcard queries) but still each one of those points to the "original" word's Posting List.

In [6]:
def reverseWord(word):
  return word[::-1]

class Index:
    """
    An Index needs to have two kind of dictionaries:
    • A normal one in which we store the words and their 
      associated posting list (so a Term object).
    • A reversed one in which words are stored starting from 
      last letter, this is useful when dealing with leading 
      wildcard queries.
    The Index can be built from a corpus of documents.
    """
    def __init__(self):
        self._dictionary = []
        self._reversed_dictionary = []
    
    @classmethod
    def fromCorpus(cls, corpus):
      """
      Creates an Index starting from a given corpus.
      For each document we extract a list of tokens and 
      for each of those, if the token was already present then 
      we just merge the term related Posting Lists, otherwise 
      we add it to the dictionary, add the end of word symbol, 
      perform all possible rotation of the word and insert them 
      into the dictionary.
      """
      intermediate_dict = {}
      for docID, document in enumerate(corpus):
        tokens = tokenize(document)
        for position, token in enumerate(tokens):
          term = Term.fromDocID(token, docID, position)
          try:
            intermediate_dict[token].merge(term)
          except KeyError:
            intermediate_dict[token] = term
            token += '$'
            rotations = [token[idx:]+token[:idx] for idx in range(len(token))]
            for rotation in rotations:
              term = Term.fromPostingList(rotation, term.posting_list)
              intermediate_dict[rotation] = term
      idx = cls()
      idx._dictionary = sorted(intermediate_dict.values())
      idx._reversed_dictionary = sorted(intermediate_dict.values(), key = lambda x : reverseWord(x.term))
      return idx
    
    def __getitem__(self, key):
      key_term = Term(term=key)
      bin_search_index = bisect.bisect_left(self._dictionary, key_term)
      if bin_search_index < len(self._dictionary) and key_term == self._dictionary[bin_search_index]:
        return self._dictionary[bin_search_index].posting_list

      raise KeyError(f"{key} is not present in the index!")
        
    def __repr__(self):
        return "A dictionary with " + str(len(self._dictionary)) + " terms"

## Reading the Corpus

The **corpus** we decided to work on is made of different posts taken from the famous subreddit [r/nba](https://www.reddit.com/r/nba/). This has been retrieved by using Reddit's built in API .json functionality to scrape post data and following the procedure shown [here](https://github.com/templecm4y/project-reddit-nlp/blob/master/P3-Reddit-NLP-get-subreddit-data.ipynb). 

This collection contains 2315 posts and from each one of we extract the title and description.

In [7]:
class NbaPostDescription:
    """
    A NBA post has a title and a relative text associated,
    which for convenience we will call description.
    """
    def __init__(self, title:str, description:str):
        self.title = title
        self.description = description
    
    def __repr__(self):
        return self.title

def readNbaDescription():
    """
    Returns the corpus of documents taken from r/NBA subreddit 
    posts. These are contained in the NBA_data.csv file.
    """
    filename = "NBA_data.csv"
    with open (filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        # Skip header line
        next(csv_reader)
        corpus = []
        for row in csv_reader:
            description = row[0] if len(row[1]) == 0 else row[1]
            post = NbaPostDescription(title=row[0], description=description)
            corpus.append(post)
    
    return corpus

## Edit Distance

Spelling Correction can be implemented in different ways, here we decided to go with the **Edit (or Levenshtein) Distance**. This technique allows to measure the number of insertion, deletion or updates that we have to do if we want transform a word into another one.

In [8]:
def editDistance(target:str, source:str):
    """
    In order to implement spelling correction we can use edit
    distance. Levenshtein distance is the one that is applied
    to compute the "distance" between two strings, and allows
    for insertion, delition and substitution.
    """
    nrows = len(target) + 1
    ncols = len(source) + 1
    M = [[0] * ncols for i in range(0, nrows)]
    for i in range(0, nrows):
        M[i][0] = i
    for j in range(0, ncols):
        M[0][j] = j
    for i in range(1, nrows):
        for j in range(1, ncols):
            candidates = [M[i-1][j] + 1, M[i][j-1] + 1]
            if target[i-1] == source[j-1]:
                candidates.append(M[i-1][j-1])
            else:
                candidates.append(M[i-1][j-1] + 1)
            M[i][j] = min(candidates)
    return M[-1][-1]

def findNearest(word:str, dictionary, keep_first=False):
    """
    Returns the nearest word with respect to the mispelled one.
    The parameter keep_first can be set to True if we want to 
    look for similar words present in the dictionary that start 
    with the same character. This should enhance the overall
    performances of the algorithm.
    """
    if keep_first:
        dictionary = list(filter(lambda w: w[0] == word[0], dictionary))
    distances = map(lambda x: editDistance(word, x), dictionary) 
    return min(zip(distances, dictionary))[1]

## IR System

Finally we can implement our **Boolean Retrieval System**. This allows to answer all the different kind of queries that we talked about at the beginning.

In [9]:
class IRSystem:
    """
    An Information Retrieval System is composed by a 
    collection of documents and by an index.
    """
    def __init__(self, corpus:list, index:'Index'):
        self._corpus = corpus
        self._index = index
        
    @classmethod
    def fromCorpus(cls, corpus:list):
        index = Index.fromCorpus(corpus)
        return cls(corpus, index)
    
    def spellingCorrection(self, words:list):
        """
        Corrects the spelling of a word persent in a list, 
        if mispelled, and return a list of the retrieved 
        posting lists.
        """
        postings = []
        for word in words:
            try:
                res = self._index[word]
            except KeyError:
                dictionary = [t.term for t in self._index._dictionary if "$" not in t.term]
                substitute = findNearest(word, dictionary, keep_first=False)
                print(f"{word} not found. Did you mean {substitute}?")
                res = self._index[substitute]
            postings.append(res)
        
        return postings
    
    def normalizeAndCorrect(self, words, spelling_correction):
        norm_words = map(normalize, words)
        postings = self.spellingCorrection(norm_words) if spelling_correction else map(lambda w: self._index[w], norm_words)
        
        return postings
    
    ############--------------BooleanQueries--------------############

    def answerOrQuery(self, words:list, spelling_correction=False):
        """
        Answer an or query.
        """
        postings = self.normalizeAndCorrect(words, spelling_correction)
        posting_list = reduce(lambda x, y: x.union(y), postings)

        return posting_list.getFromCorpus(self._corpus)
    
    def answerAndQuery(self, words:list, spelling_correction=False):
        """
        Answer an and query.
        """
        postings = self.normalizeAndCorrect(words, spelling_correction)
        posting_list = reduce(lambda x, y: x.intersection(y), postings)

        return posting_list.getFromCorpus(self._corpus)
    
    def answerNotQuery(self, words:list, spelling_correction=False):
        """
        Answer a not query.
        """
        postings = self.normalizeAndCorrect(words, spelling_correction)
        posting_list = reduce(lambda x, y: x.intersection(y), postings)
        negate_posting_list = [posting for posting in self._index._posting_list if posting not in posting_list]
        negate_posting_list = PostingList.fromPostingList(negate_posting_list)

        return negate_posting_list.getFromCorpus(self._corpus)
    
    def answerQuery(self, words:list, operator:str, previous_postings:'PostingList' = None, next_postings:'PostingList' = None, spelling_correction = False, apply_all_not = False):
        """
        Answer a general query.
        Parameter apply_all_not is used when we answer queries
        with parentheses, and so we need to distribute the
        not operator to every element inside them.
        """
        if len(words) == 1:
            norm_w0 = normalize(words[0])
            next_postings = self.spellingCorrection([norm_w0])[0] if spelling_correction else self._index[norm_w0]
        elif len(words) > 1:
            norm_w0 = normalize(words[0])
            norm_w1 = normalize(words[1])
            previous_postings = self.spellingCorrection([norm_w0])[0] if spelling_correction else self._index[norm_w0]
            next_postings = self.spellingCorrection([norm_w1])[0] if spelling_correction else self._index[norm_w1]
                    
        if operator == 'AND':
            return previous_postings.intersection(next_postings)
        elif operator == 'OR':
            return previous_postings.union(next_postings)
        elif operator == 'NOT':
            if not apply_all_not:
                negate_posting_list = [posting for posting in previous_postings if posting not in next_postings]
                return PostingList.fromPostingList(negate_posting_list)
            else:
                negate_posting_list = [posting for posting in next_postings if posting not in previous_postings]
                return PostingList.fromPostingList(negate_posting_list)
    
    ############--------------PhraseQueries--------------############

    def answerPhraseQuery(self, words:list, spelling_correction = False):
        """
        Answer a Phrase query.
        """
        postings = list(self.normalizeAndCorrect(words, spelling_correction))
        result_posting_list = postings[0]
        for i in range(1, len(postings)):
            result_posting_list = result_posting_list.positionalSearch(postings[i], i)
        return result_posting_list, result_posting_list.getFromCorpus(self._corpus)
    
    def answerPhraseQueryK(self, term1:str, term2:str, k:int, spelling_correction=False):
        """
        Answer a Phrase query in which we specify that
        maximum distance between two terms can be at most k.
        """
        norm_w0 = normalize(term1)
        norm_w1 = normalize(term2)
        first_posting_list = self.spellingCorrection([norm_w0])[0] if spelling_correction else self._index[norm_w0]
        second_posting_list = self.spellingCorrection([norm_w1])[0] if spelling_correction else self._index[norm_w1]
        result_posting_list = []
        for i in range(1, k+2):
            result_posting_list.append(first_posting_list.positionalSearch(second_posting_list, i))
        result_posting_list = reduce(lambda x, y: x.union(y), result_posting_list)
        return result_posting_list, result_posting_list.getFromCorpus(self._corpus)
    
    ############--------------WildcardQueries--------------############

    def answerTrailingWildcard(self, wildcard:str):
        """
        Answer a Trailing Wildcard query.
        """
        posting_lists = [term.posting_list for term in self._index._dictionary if term.term.startswith(wildcard)]
        posting_list = reduce(lambda x, y: x.union(y), posting_lists)
        return posting_list.getFromCorpus(self._corpus) 
    
    # def answerTrailingWildcard(self, wildcard:str):
    #     """
    #     Answer a Trailing Wildcard query.
    #     """
    #     posting_list = [term.posting_list for term in self._index._dictionary if re.search(wildcard+'.+', term.term)]
    #     posting_list = reduce(lambda x, y: x.union(y), posting_list)
    #     return posting_list.getFromCorpus(self._corpus)
    

    def answerLeadingWildcard(self, wildcard:str):
        """
        Answer a Leading Wildcard query.
        The wildcard is reversed so we can search properly 
        in the reversed dictionary.
        """
        posting_list = [term.posting_list for term in self._index._reversed_dictionary if term.term.endswith(wildcard)]
        posting_list = reduce(lambda x, y: x.union(y), posting_list)
        return posting_list.getFromCorpus(self._corpus)
    
    def answerMultipleWildcards(self, word):
        """
        Answer a multiple Wildcard query.
        """
        word += "$"
        last_wildcard_position = word.rfind("*")
        first_wildcard_position = word.index("*")
        simplified_word = word[:first_wildcard_position] + word[last_wildcard_position:]
        i = simplified_word.index("*")
        simplified_wildcard = normalize(simplified_word[i+1:] + simplified_word[:i])
        simplified_query_matching_terms = [term for term in self._index._dictionary if term.term.startswith(simplified_wildcard)]
        unfolded = simplified_wildcard + word[first_wildcard_position:last_wildcard_position+1]
        unfolded = unfolded.replace("*", "\S*").replace("$", "\$")
        posting_lists = [term.posting_list for term in simplified_query_matching_terms if re.search(unfolded, term.term)]
        posting_list = reduce(lambda x, y: x.union(y), posting_lists)
        return posting_list.getFromCorpus(self._corpus)         

## Boolean Queries

In [10]:
def printResult(result:list):
    """
    Print the result of a query.
    """
    for title in result:
        print(title)

# Possible Boolean operators.
operators = ['AND', 'OR', 'NOT']

In [11]:
def queryOR(ir:'IRSystem', text:str, spelling_correction=False):
    """
    Executes or query.
    """
    words = text.split()
    answer = ir.answerOrQuery(words, spelling_correction)
    printResult(answer)

    return answer

def queryAND(ir:'IRSystem', text:str, spelling_correction=False):
    """
    Executes and query.
    """
    words = text.split()
    answer = ir.answerAndQuery(words, spelling_correction)
    printResult(answer)

    return answer

def queryNOT(ir:'IRSystem', text:str, spelling_correction=False):
    """
    Executes not query.
    """
    words = text.split()
    answer = ir.answerNotQuery(words, spelling_correction)
    printResult(answer)

    return answer

def query(ir:'IRSystem', text:str, spelling_correction=False):
    """
    Executes a general Boolean query.
    """
    words = text.split()
    if len(words) == 1:
        return queryAND(ir, text, spelling_correction)
    for i, word in enumerate(words):
        if word in operators:
            if i != 1:
                posting_list = ir.answerQuery(words=[words[i+1]], previous_postings=posting_list, 
                                              operator=word, spelling_correction=spelling_correction)
            else:
                posting_list = ir.answerQuery(words=[words[i-1], words[i+1]], 
                                              operator=word, spelling_correction=spelling_correction)
    answer = posting_list.getFromCorpus(ir._corpus)
    printResult(answer)

    return answer

def queryWithParentheses(ir:'IRSystem', text:str, spelling_correction=False):
    """
    Executes a query in which search terms and their operators 
    are enclosed in parentheses to specify the order in 
    which they are interpreted.
    Information within parentheses is read first, and then 
    information outside parentheses is read next.
    """
    text = text.replace("(", "( ").replace(")", " )")
    words = text.split()
    # If the query is just something like "(word)"
    if len(words) == 3:
        return queryAND(ir, words[1], spelling_correction)
    
    # To be more general we can insert a parenthesis at the
    # beginning and at the end of the list of words.
    words.append(")")
    words.insert(0,"(")
    # Now we need to take care of parentheses indices. 
    # Those can be stored inside a dictionary, the value
    # of the corresponding key will be a list of the two
    # indices. In the meanwhile we can check that there 
    # are no unbalanced parentheses.
    parentheses_indices = {}
    parentheses_counter = 0
    tot_parentheses = 0
    right_par = 0
    nested_par = 0
    for position, word in enumerate(words):
        if word == "(":
            parentheses_counter += 1
            tot_parentheses += 1
            parentheses_indices[f"par_{tot_parentheses}"] = [position]
        elif word == ")":
            right_par += 1
            parentheses_counter -= 1
            if tot_parentheses == right_par:
                parentheses_indices[f"par_1"].append(position)
            elif tot_parentheses == (right_par+1):
                parentheses_indices[f"par_{tot_parentheses-nested_par}"].append(position)
                if nested_par != 0:
                    nested_par = 0
            else:
                # nested parentheses
                parentheses_indices[f"par_{tot_parentheses-nested_par}"].append(position)
                nested_par += 1
    
    if parentheses_counter != 0:
        print("Parentheses are not balanced! Reformulate the query!")
        return None
        
    while tot_parentheses > 0:
        left_par_index = parentheses_indices[f"par_{tot_parentheses}"][0]
        right_par_index = parentheses_indices[f"par_{tot_parentheses}"][1]
        for i, word in enumerate(words[left_par_index+1:right_par_index]):
            if word in operators:
                if i!= 1:
                    second_term = words[left_par_index+1 + i+1]
                    if type(second_term) == PostingList:
                        posting_list = ir.answerQuery(words=[], previous_postings=posting_list, next_postings=second_term,
                                                      operator=word, spelling_correction=spelling_correction)
                    elif type(second_term) == str:
                        posting_list = ir.answerQuery(words=[second_term], previous_postings=posting_list,
                                                      operator=word, spelling_correction=spelling_correction)
                else:
                    first_term = words[left_par_index+1 + i-1]
                    second_term = words[left_par_index+1 + i+1]
                    if type(first_term)==type(second_term)==str:
                        posting_list = ir.answerQuery(words=[first_term, second_term], 
                                                      operator=word, spelling_correction=spelling_correction)
                    elif type(first_term)==str and type(second_term)==PostingList:
                        posting_list = ir.answerQuery(words=[first_term], previous_postings=second_term, 
                                                      operator=word, spelling_correction=spelling_correction, apply_all_not=True)
                    elif type(first_term)==PostingList and type(second_term)==str:
                        posting_list = ir.answerQuery(words=[second_term], previous_postings=first_term, 
                                                      operator=word, spelling_correction=spelling_correction)
                    elif type(first_term)==PostingList and type(second_term)==PostingList:
                        posting_list = ir.answerQuery(words=[], previous_postings=first_term, next_postings=second_term,
                                                      operator=word, spelling_correction=spelling_correction)
        words[left_par_index] = posting_list
        # Remove already processed terms and parentheses
        del words[left_par_index+1:right_par_index+1]
        width_parentheses = right_par_index-left_par_index
        tot_parentheses -= 1
        # Decrease the index of the remaining parentheses
        for num_parenthesis in range(tot_parentheses, 0, -1):
            if parentheses_indices[f"par_{num_parenthesis}"][1] > parentheses_indices[f"par_{tot_parentheses+1}"][0]:
                parentheses_indices[f"par_{num_parenthesis}"][1] -= width_parentheses
    
    answer = posting_list.getFromCorpus(ir._corpus)
    printResult(answer)

    return answer

## Phrase Queries

In [12]:
def phraseQuery(ir:'IRSystem', text:str, spelling_correction=False):
    """
    Answer a Phrase query.
    """
    words = text.split()
    posting_list, answer = ir.answerPhraseQuery(words, spelling_correction)
    printResult(answer)

    return posting_list, answer

def phraseQueryKDistance(ir:'IRSystem', text:str, spelling_correction=False):
    """
    Answer a Phrase query of the form "term1 /k term2", where
    k represents the maximum distance between the two terms.
    """
    assert len(text.split()) == 3 and text.split()[1][0]=='/', "Give me a query that follows this format –––> term1 /k term2"
    words = text.split()
    posting_list, answer = ir.answerPhraseQueryK(term1=words[0], term2=words[2], k=int(words[1][1]), 
                                                 spelling_correction=spelling_correction)
    printResult(answer)

    return posting_list, answer

## Wildcard Queries

In [13]:
def trailingWildcardQuery(ir:'IRSystem', text:str):
    """
    Executes a Trailing Wildcard query, e.g. "ca*".
    """
    wildcard = text.split('*')[0]
    wildcard = normalize(wildcard)
    answer = ir.answerTrailingWildcard(wildcard)
    printResult(answer)

    return answer

def leadingWildcardQuery(ir:'IRSystem', text:str):
    """
    Executes a Leading Wildcard query, e.g. "*ar".
    """
    wildcard = text.split('*')[1]
    wildcard = normalize(wildcard)
    answer = ir.answerLeadingWildcard(wildcard)
    printResult(answer)

    return answer

def generalWildcardQuery(ir:'IRSystem', text:str):
    """
    Executes a Leading Wildcard query, e.g. "ba*an".
    Simply rotate the word until we reach the same
    situation as a Trailing Wildcard.
    """
    text += "$"
    i = text.index("*")
    wildcard = text[i+1:] + text[:i]
    wildcard = normalize(wildcard)
    answer = ir.answerTrailingWildcard(wildcard)
    printResult(answer)

    return answer

def multipleWildcardQuery(ir:'IRSystem', text:str):
    """
    Executes a Multiple Wildcard query, e.g. "s*mp*n".
    """
    answer = ir.answerMultipleWildcards(text)
    printResult(answer)

    return answer

## Testing entirely the System

In order to save and load the Index to the disk we're going to use _**Pickle**_. This module is used for serializing and de-serializing a Python object structure, i.e. Pickle can convert an object into a character stream before writing it to file and then can re-convert the byte stream into the original Python object.

In [14]:
def saveIndexToDisk(index:'Index', filename:str):
    """
    Saves an Index to the disk.
    """
    start = time.perf_counter()
    with open(filename, 'wb') as f:
        pickle.dump(index, f)
    elapsed = time.perf_counter() - start
    print(f"Saving the Index to the disk took {elapsed:.3}s")

def loadIndexFromDisk(filename:str):
    """
    Loads an Index from disk.
    """
    start = time.perf_counter()
    with open(filename, 'rb') as f:
        index = pickle.load(f)
    elapsed = time.perf_counter() - start
    print(f"Loading the Index from disk took {elapsed:.3}s")

    return index

Now we can read the corpus, get the Index and finally initialize our Information Retrieval System.

In [15]:
corpus = readNbaDescription()

In [16]:
filename = "indexPickled"
if os.path.isfile(filename):
    index = loadIndexFromDisk(filename)
else:
    start = time.perf_counter()
    index = Index.fromCorpus(corpus)
    elapsed = time.perf_counter() - start
    print(f"Building the Index took {elapsed:.3}s")
    saveIndexToDisk(index, filename)

Loading the Index from disk took 0.45s


In [17]:
ir = IRSystem(corpus, index)

### Testing Queries

_**For each test it is given the elapsed time.** This can be used as an estimation of speed performances of our System._


First we can test some trivial queries using AND, OR, NOT operators.
This can be easily done by comparing the result of the query obtained from the correspondent query function (e.g. _queryAND, queryOR, queryNOT_) and the answer manually retrieved from the index (in this case using _getAnswerFromIdx_ function).

#### AND queries

In [18]:
def getAnswerFromIdx(ir:'IRSystem', text:str):
    return set(ir._index[normalize(text)].getFromCorpus(ir._corpus))

def verifyAnswers(answer1:list, answer2:set):
    assert set(answer1) == answer2, "Found a mismatch between the two answers!!"
    print("The two answers match!")

In [19]:
start = time.perf_counter()
answer = queryAND(ir, "Lebron James Miami")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

2019-20 NBA Free Agents and Team Roster Tracker
National TV Schedule Predictions for Major Dates
If we are truly in the Era of "The Duo's" now, it's time to remake NBA Jam.
[Arthur] During the playoffs, and even the Finals, sources indicate Kawhi and Dennis Robertson were focused on LAC, even as Toronto’s championship run unfolded.The title closed the gap. By Thursday night sources say the LAC were telling people in the league they thought Kawhi was going back to TOR
Lebron James and Anthony Davis have both ranked in the top 10 in PER for the last 6 years.
Why LeBron would sign with every NBA team
Fun Fact: Lebron James has been swept in every finals he hasn’t had James Jones as a teammate.
[OC] How Every Team Could Trade For Russell Westbrook
[OC] How close has each team without an NBA championship come to winning one?

Elapsed time was 1.68ms


In [20]:
answer_lebron = getAnswerFromIdx(ir, "Lebron")
answer_james = getAnswerFromIdx(ir, "James")
answer_miami = getAnswerFromIdx(ir, "miami")
composed_answer = answer_lebron.intersection(answer_james).intersection(answer_miami)
verifyAnswers(answer, composed_answer)

The two answers match!


In [21]:
start = time.perf_counter()
answer = queryAND(ir, "Kawhi philadelphia")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

[OC] How the Nets and Clippers set themselves up to be the biggest winners of 2019 Free Agency
2019-20 NBA Free Agents and Team Roster Tracker
[OC] Fanbase Happiness Power Rankings
National TV Schedule Predictions for Major Dates
Quick look at how every team has changed so far this offseason
[Arthur] During the playoffs, and even the Finals, sources indicate Kawhi and Dennis Robertson were focused on LAC, even as Toronto’s championship run unfolded.The title closed the gap. By Thursday night sources say the LAC were telling people in the league they thought Kawhi was going back to TOR
[OC] How Every Team Could Trade For Russell Westbrook

Elapsed time was 1.03ms


In [22]:
answer_kawhi = getAnswerFromIdx(ir, "kawhi")
answer_phila = getAnswerFromIdx(ir, "Philadelphia")
composed_answer = answer_kawhi.intersection(answer_phila)
verifyAnswers(answer, composed_answer)

The two answers match!


#### OR queries

In [23]:
start = time.perf_counter()
answer = queryOR(ir, "Cleveland Miami Lakers")
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")

Does Paul George's recent trade request confirm or invalidate the notion that market size and market location still matters to players' title aspirations?
The record for number of combined wins by both LA teams since the move to Staples is 101, set during the 2013 season. Do they break that record this season? If yes, by how much?
Adam Morrison was the better basketball player than Brian Scalabrine
What exactly happened between Jerry West and the Lakers?
Can we pour one out for the Suns? They have to face the Lakers, Clippers and Warriors 4x next year
What teams do you think will under perform compared to expectations this season?
2013 Miami Heat vs 2017 Golden State Warriors...
Point guard talk
[Jacob Rude] LeBron cites a handful of strong defensive performances, all in the 2nd half of games he didn't play in. He then joked that "maybe I need to sit my ass down" for the team to be strong defensively.
The biggest red flag for each LA team.
Does Paul George's recent trade request confir

In [24]:
answer_cleveland = getAnswerFromIdx(ir, "cleveland")
answer_lakers = getAnswerFromIdx(ir, "lakers")
composed_answer = answer_miami.union(answer_cleveland).union(answer_lakers)
verifyAnswers(answer, composed_answer)

The two answers match!


#### NOT queries

In [25]:
start = time.perf_counter()
answer = queryNOT(ir, "Lebron")
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")

Game Threads Index + Daily Discussion (July 04, 2019)
2019 NBA Free Agent Tracker
[Haynes] Free agent guard Quinn Cook has reached an agreement with the Los Angeles Lakers on a two-year, $6 million deal, league sources tell Yahoo Sports.
[Wojnarowski] Presti pursued a package of Russell Westbrook with George to the Raptors -- with the NBA's Most Improved Player, forward Pascal Siakam as the centerpiece of a deal -- and Ujiri balked, league sources said.
(Shelburne in ESPN piece) Still, Leonard's recruiting efforts caught George by surprise. Said one source close to George, "For a quiet guy, he's a hell of a recruiter."
[Wojnarowski] Oklahoma City is trading All-Star Paul George to the Los Angeles Clippers for a record-setting collection of draft choices, league sources tell ESPN.
The Nets and Clippers, both viewed as "little brother" franchises in their own cities, just signed the top two free agents in the same week.
[Wojnarowski] Kawhi Leonard has been recruiting Paul George to find 

In [26]:
composed_answer= set(corpus).difference(answer_lebron)
verifyAnswers(answer,composed_answer)

The two answers match!


In [27]:
start = time.perf_counter()
answer_not = queryNOT(ir, "Magic Johnson")
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")

Game Threads Index + Daily Discussion (July 04, 2019)
2019 NBA Free Agent Tracker
[Haynes] Free agent guard Quinn Cook has reached an agreement with the Los Angeles Lakers on a two-year, $6 million deal, league sources tell Yahoo Sports.
[Wojnarowski] Presti pursued a package of Russell Westbrook with George to the Raptors -- with the NBA's Most Improved Player, forward Pascal Siakam as the centerpiece of a deal -- and Ujiri balked, league sources said.
(Shelburne in ESPN piece) Still, Leonard's recruiting efforts caught George by surprise. Said one source close to George, "For a quiet guy, he's a hell of a recruiter."
[Wojnarowski] Oklahoma City is trading All-Star Paul George to the Los Angeles Clippers for a record-setting collection of draft choices, league sources tell ESPN.
The Nets and Clippers, both viewed as "little brother" franchises in their own cities, just signed the top two free agents in the same week.
[Wojnarowski] Kawhi Leonard has been recruiting Paul George to find 

In [28]:
answer_magic = getAnswerFromIdx(ir, "magic")
answer_johnson = getAnswerFromIdx(ir, "johnson")
composed_answer = set(corpus).difference(answer_magic.intersection(answer_johnson))
verifyAnswers(answer_not, composed_answer)

The two answers match!


#### Spelling Correction 
Let's try to mispell some words and see what happens:

In [29]:
start = time.perf_counter()
answer = queryNOT(ir, "Megic Jahnsen", spelling_correction=True)
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")

megic not found. Did you mean magic?
jahnsen not found. Did you mean johnson?
Game Threads Index + Daily Discussion (July 04, 2019)
2019 NBA Free Agent Tracker
[Haynes] Free agent guard Quinn Cook has reached an agreement with the Los Angeles Lakers on a two-year, $6 million deal, league sources tell Yahoo Sports.
[Wojnarowski] Presti pursued a package of Russell Westbrook with George to the Raptors -- with the NBA's Most Improved Player, forward Pascal Siakam as the centerpiece of a deal -- and Ujiri balked, league sources said.
(Shelburne in ESPN piece) Still, Leonard's recruiting efforts caught George by surprise. Said one source close to George, "For a quiet guy, he's a hell of a recruiter."
[Wojnarowski] Oklahoma City is trading All-Star Paul George to the Los Angeles Clippers for a record-setting collection of draft choices, league sources tell ESPN.
The Nets and Clippers, both viewed as "little brother" franchises in their own cities, just signed the top two free agents in the s

We can notice that spelling correction takes more time than executing a correct query.

In [30]:
verifyAnswers(answer, set(answer_not))

The two answers match!


#### Boolean Queries

First we can test Boolean queries **without parentheses**:

In [31]:
start = time.perf_counter()
answer = query(ir, "lebron AND wade")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

2019-20 NBA Free Agents and Team Roster Tracker
PSA: Not every Elite team is a "super team"
The last 9 All-Star Game MVPs have all played for California teams.
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
Lebron James and Anthony Davis have both ranked in the top 10 in PER for the last 6 years.
Carmelo Anthony was never a top 5 player. Not even once.
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
I put NBA players into the Wu Tang name generator
[OC] Top 100 Scorers in NBA Finals History in Terms of Points Per Game
[OC] How My 3-Year-Old Says the Name of Every 2019 NBA All-Star

Elapsed time was 1.48ms


In [32]:
answer_wade = getAnswerFromIdx(ir, "wade")
composed_answer = answer_lebron.intersection(answer_wade)
verifyAnswers(answer, composed_answer)

The two answers match!


In [33]:
start = time.perf_counter()
answer = query(ir, "lebron OR wade")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

Lets say neither LA team wins a championship, and their contending windows close. Which would be considered the worst, the PG trade or the AD trade?
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
Is Kawhi + PG the most overlap we've had (in terms of play style and between two superstar players since Duncan/Robinson or even earlier?
Is having Lebron really worth the trouble?
National TV Schedule Predictions for Major Dates
Is it 2018 or 2008?
[Trudell] Asked about watching his son play in Saturday’s showcase, LeBron James said he still thinks about playing w/him one day. He said Bronny’s dream is to play in NBA: “He has my support and my blueprint. With health and a little bit of luck, that would be the ultimate thing.”
The top long term players in the NBA, according to 538: 1-Harden, 2-Giannis, 3-Luka, 4-Simmons, 5-Jokic, 6-Towns, 7-AD, 8-Embiid, 9-Lonzo, 10-Tatum
All-NBA teams based solely off of player headshots
PSA

In [34]:
composed_answer = answer_lebron.union(answer_wade)
verifyAnswers(answer, composed_answer)

The two answers match!


In [35]:
start = time.perf_counter()
answer = query(ir, "lebron AND wade OR bosh")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

I put NBA players into the Wu Tang name generator
[OC] How My 3-Year-Old Says the Name of Every 2019 NBA All-Star
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
[OC] Top 100 Scorers in NBA Finals History in Terms of Points Per Game
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
Lebron James and Anthony Davis have both ranked in the top 10 in PER for the last 6 years.
3 years ago, the Miami Heat rolled out a lineup with 5 left handed players
Is this the most compatible team LeBron has ever played with?
Carmelo Anthony was never a top 5 player. Not even once.
The last 9 All-Star Game MVPs have all played for California teams.
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
PSA: Not every Elite team is a "super team"
2019-20 NBA Free Agents and Team Roster Tracker
PSA: Not every Elite team is a "super team"

Elapsed time was 0.849ms


In [36]:
answer_bosh = getAnswerFromIdx(ir, "bosh")
composed_answer = answer_lebron.intersection(answer_wade).union(answer_bosh)
verifyAnswers(answer, composed_answer)

The two answers match!


In [37]:
start = time.perf_counter()
answer = query(ir, "lebron OR wade AND bosh")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

PSA: Not every Elite team is a "super team"
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
PSA: Not every Elite team is a "super team"

Elapsed time was 0.647ms


In [38]:
composed_answer = answer_lebron.union(answer_wade).intersection(answer_bosh)
verifyAnswers(answer, composed_answer)

The two answers match!


In [39]:
start = time.perf_counter()
answer = query(ir, "lebron AND wade NOT cleveland")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

PSA: Not every Elite team is a "super team"
The last 9 All-Star Game MVPs have all played for California teams.
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
Lebron James and Anthony Davis have both ranked in the top 10 in PER for the last 6 years.
Carmelo Anthony was never a top 5 player. Not even once.
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
I put NBA players into the Wu Tang name generator
[OC] Top 100 Scorers in NBA Finals History in Terms of Points Per Game
[OC] How My 3-Year-Old Says the Name of Every 2019 NBA All-Star

Elapsed time was 1.9ms


In [40]:
composed_answer = answer_lebron.intersection(answer_wade).difference(answer_cleveland)
verifyAnswers(answer, composed_answer)

The two answers match!


In [41]:
start = time.perf_counter()
answer = query(ir, "lebron NOT wade AND Cleeeveland", spelling_correction=True)
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")

cleeeveland not found. Did you mean cleveland?
How good was LeBron defensively in the 2014-2015 and 2015-2016 seasons(RS + Playoffs), Compared to his peak Miami years?
The Domino Effect of the James Harden Trade
Why LeBron would sign with every NBA team
Is having Lebron really worth the trouble?
One thing that pissed me off about the draft
Fun Fact: Lebron James has been swept in every finals he hasn’t had James Jones as a teammate.
[OC] In-depth analsis of Dwight Howard’s run to the 2009 NBA Finals
Would Devin Booker be looked at similarly to Kyrie Irving if the Suns had added a Lebron-like player?
The NBA is the only major sport where players can be bigger than the team
Kyrie once purposely avoided and ignored calls from a Cavs assistant coach who was assigned to train him. After two weeks, the coach tracked Kyrie down in Miami, where Kyrie admitted he was simply testing the coach's motives
[OC] How Every Team Could Trade For Russell Westbrook
What if Kawhi pulls off a LeBron and hop

In [42]:
composed_answer = answer_lebron.difference(answer_wade).intersection(answer_cleveland)
verifyAnswers(answer, composed_answer)

The two answers match!


Now we can try to test Boolean queries by **using parentheses**.

In [43]:
start = time.perf_counter()
answer = queryWithParentheses(ir, "(lebron)")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

[Discussion] The NBA "superteam" era has temporarily halted for the "duo" era
Can we just appreciate Kawhi's decision as neutral fans?
2019-20 NBA Free Agents and Team Roster Tracker
A Lakers-Sixers final would be a hell of a frontcourt showdown
Chicago Bulls have quietly become obsessed with advanced analytics and had a really good low key offseason.
Some funny Chinese nicknames of NBA players and how these name came
In his age 34/35 season, Michael Jordan made $33,140,000; accounting for 123% of the salary cap
The Lakers have done a commendable job filling out their roster.
3 Reasons why Kevin Durant chose number 7.
National TV Schedule Predictions for Major Dates
The biggest red flag for each LA team.
It's the offseason. Here is my top 100 player list.
The Lakers should unironically start Caruso
If we are truly in the Era of "The Duo's" now, it's time to remake NBA Jam.
PSA: Not every Elite team is a "super team"
The Lakers have managed to dramatically improve the biggest weakness o

In [44]:
verifyAnswers(answer, answer_lebron)

The two answers match!


In [45]:
start = time.perf_counter()
answer = queryWithParentheses(ir, "(lebron AND wade) NOT cleveland")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

PSA: Not every Elite team is a "super team"
The last 9 All-Star Game MVPs have all played for California teams.
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
Lebron James and Anthony Davis have both ranked in the top 10 in PER for the last 6 years.
Carmelo Anthony was never a top 5 player. Not even once.
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
I put NBA players into the Wu Tang name generator
[OC] Top 100 Scorers in NBA Finals History in Terms of Points Per Game
[OC] How My 3-Year-Old Says the Name of Every 2019 NBA All-Star

Elapsed time was 0.802ms


In [46]:
composed_answer = answer_lebron.intersection(answer_wade).difference(answer_cleveland)
verifyAnswers(answer, composed_answer)

The two answers match!


In [47]:
start = time.perf_counter()
answer = queryWithParentheses(ir, "lebron AND (wade NOT cleveland)")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

PSA: Not every Elite team is a "super team"
The last 9 All-Star Game MVPs have all played for California teams.
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
Lebron James and Anthony Davis have both ranked in the top 10 in PER for the last 6 years.
Carmelo Anthony was never a top 5 player. Not even once.
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
I put NBA players into the Wu Tang name generator
[OC] Top 100 Scorers in NBA Finals History in Terms of Points Per Game
[OC] How My 3-Year-Old Says the Name of Every 2019 NBA All-Star

Elapsed time was 0.698ms


In [48]:
composed_answer = answer_lebron.intersection((answer_wade).difference(answer_cleveland))
verifyAnswers(answer, composed_answer)

The two answers match!


In [49]:
start = time.perf_counter()
answer = queryWithParentheses(ir, "(lebron AND wade) NOT (cleveland OR lakers)")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
Lebron James and Anthony Davis have both ranked in the top 10 in PER for the last 6 years.
Carmelo Anthony was never a top 5 player. Not even once.
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
I put NBA players into the Wu Tang name generator
[OC] Top 100 Scorers in NBA Finals History in Terms of Points Per Game
[OC] How My 3-Year-Old Says the Name of Every 2019 NBA All-Star

Elapsed time was 1.59ms


In [50]:
composed_answer = answer_lebron.intersection(answer_wade).difference(answer_cleveland.union(answer_lakers))
verifyAnswers(answer, composed_answer)

The two answers match!


In [51]:
start = time.perf_counter()
answer = queryWithParentheses(ir, "lebron AND (wade AND (bosh OR kawhi) NOT magic) OR lakers")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

The record for number of combined wins by both LA teams since the move to Staples is 101, set during the 2013 season. Do they break that record this season? If yes, by how much?
Adam Morrison was the better basketball player than Brian Scalabrine
What exactly happened between Jerry West and the Lakers?
Can we pour one out for the Suns? They have to face the Lakers, Clippers and Warriors 4x next year
What teams do you think will under perform compared to expectations this season?
Point guard talk
[Jacob Rude] LeBron cites a handful of strong defensive performances, all in the 2nd half of games he didn't play in. He then joked that "maybe I need to sit my ass down" for the team to be strong defensively.
The biggest red flag for each LA team.
Does Paul George's recent trade request confirm or invalidate the notion that market size and market location still matters to players' title aspirations?
PSA: Not every Elite team is a "super team"
What will be the clippers and lakers starting lineu

In [52]:
composed_answer = answer_lebron.intersection(answer_wade.intersection(answer_bosh.union(answer_kawhi).difference(answer_magic))).union(answer_lakers)
verifyAnswers(answer, composed_answer)

The two answers match!


In [53]:
start = time.perf_counter()
answer = queryWithParentheses(ir, "lebbbbron AND (wade AND (bosgh OR kawahi) NOT magic) OR lakerzs", spelling_correction=True)
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")

bosgh not found. Did you mean bosh?
kawahi not found. Did you mean kawhi?
lebbbbron not found. Did you mean lebron?
lakerzs not found. Did you mean lakers?
The record for number of combined wins by both LA teams since the move to Staples is 101, set during the 2013 season. Do they break that record this season? If yes, by how much?
Adam Morrison was the better basketball player than Brian Scalabrine
What exactly happened between Jerry West and the Lakers?
Can we pour one out for the Suns? They have to face the Lakers, Clippers and Warriors 4x next year
What teams do you think will under perform compared to expectations this season?
Point guard talk
[Jacob Rude] LeBron cites a handful of strong defensive performances, all in the 2nd half of games he didn't play in. He then joked that "maybe I need to sit my ass down" for the team to be strong defensively.
The biggest red flag for each LA team.
Does Paul George's recent trade request confirm or invalidate the notion that market size and 

In [54]:
composed_answer = answer_lebron.intersection(answer_wade.intersection(answer_bosh.union(answer_kawhi).difference(answer_magic))).union(answer_lakers)
verifyAnswers(answer, composed_answer)

The two answers match!


#### Phrase Queries

First we can test normal Phrase Queries:

In [55]:
def verifyPhraseQuery(text, posting_list):
    text = normalize(text)
    words = text.split()
    for posting in posting_list:
        l = len(words)
        docID = posting._docID
        positions = posting._positions
        tokens = tokenize(corpus[docID])
        counter = 0
        for position in positions:
            if tokens[position:position+l] == words:
                counter += 1
                print("Correct Positional Matching!")
        if counter == 0:
            print("No matching has been found...")

In [56]:
start = time.perf_counter()
plist, answer = phraseQuery(ir, "Cleveland Cavaliers")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")
verifyPhraseQuery("Cleveland Cavaliers", plist)

Game Threads Index + Daily Discussion (July 04, 2019)

Elapsed time was 0.61ms
Correct Positional Matching!


In [57]:
start = time.perf_counter()
plist, answer = phraseQuery(ir, "New York Knicks")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")
verifyPhraseQuery("New York Knicks", plist)

2019-20 NBA Free Agents and Team Roster Tracker

Elapsed time was 0.465ms
Correct Positional Matching!


In [58]:
start = time.perf_counter()
plist, answer = phraseQuery(ir, "Kareem Abdul Jabbar")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")
verifyPhraseQuery("Kareem Abdul Jabbar", plist)

This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception

Elapsed time was 0.433ms
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!


In [59]:
start = time.perf_counter()
plist, answer = phraseQuery(ir, "Kareem Abdul Jabbau", spelling_correction=True)
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")
verifyPhraseQuery("Kareem Abdul Jabbar", plist)

jabbau not found. Did you mean jabbar?
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception

Elapsed time was 0.384s
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!
Correct Positional Matching!


Then we can test $k$ steps Phrase Queries of the form "term1 /k term2" in which we specify the maximum distance ($k$) that can exists between the two terms.

In [60]:
start = time.perf_counter()
plist, answer = phraseQueryKDistance(ir, "Lebron /1 James")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

Fun Fact: Lebron James has been swept in every finals he hasn’t had James Jones as a teammate.
2019-20 NBA Free Agents and Team Roster Tracker

Elapsed time was 0.776ms


In [61]:
start = time.perf_counter()
plist, answer = phraseQueryKDistance(ir, "James /3 Hardden", spelling_correction=True)
print(f"\nElapsed time was {(time.perf_counter()-start):.3}s")

hardden not found. Did you mean harden?
Your way-too-early 2020 MVP prediction
GAME THREAD: Brooklyn Nets (16-7) @ Dallas Mavericks (11-11) - (December 07, 2021)
2019-20 NBA Free Agents and Team Roster Tracker

Elapsed time was 0.45s


#### Wildcard Queries

We can start by testing first **Trailing Wildcard Queries**:

In [62]:
def verifyTrailingWildcard(ir:'IRSystem', wildcard:str, answer:list):
    norm_wildcard = normalize(wildcard)
    posting_list = [term.posting_list for term in ir._index._dictionary if term.term.startswith(norm_wildcard)]
    posting_list = reduce(lambda x, y: x.union(y), posting_list)
    result = posting_list.getFromCorpus(ir._corpus)
    assert len(result) == len(answer)
    for posting in result:
        if posting not in answer:
            print("Missing Posting in the Posting List!")
    print("Trailing Wildcard Query executed correctly!")

In [63]:
start = time.perf_counter()
answer = trailingWildcardQuery(ir, "Leb*")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

Lets say neither LA team wins a championship, and their contending windows close. Which would be considered the worst, the PG trade or the AD trade?
Heres the Charlotte Hornets can win the NBA Championship next season in 2020
Is Kawhi + PG the most overlap we've had (in terms of play style and between two superstar players since Duncan/Robinson or even earlier?
Is having Lebron really worth the trouble?
National TV Schedule Predictions for Major Dates
Is it 2018 or 2008?
[Trudell] Asked about watching his son play in Saturday’s showcase, LeBron James said he still thinks about playing w/him one day. He said Bronny’s dream is to play in NBA: “He has my support and my blueprint. With health and a little bit of luck, that would be the ultimate thing.”
The top long term players in the NBA, according to 538: 1-Harden, 2-Giannis, 3-Luka, 4-Simmons, 5-Jokic, 6-Towns, 7-AD, 8-Embiid, 9-Lonzo, 10-Tatum
[Post Game Thread] The Los Angeles Clippers (13-12) defeat the Portland Trail Blazers (11-14)

In [64]:
verifyTrailingWildcard(ir, "Leb", answer)

Trailing Wildcard Query executed correctly!


Then we can continue with **Leading Wildcard Queries**:

In [65]:
def verifyLeadingWildcard(ir:'IRSystem', wildcard:str, answer:list):
    norm_wildcard = normalize(wildcard)
    posting_list = [term.posting_list for term in ir._index._reversed_dictionary if term.term.endswith(norm_wildcard)]
    posting_list = reduce(lambda x, y: x.union(y), posting_list)
    result = posting_list.getFromCorpus(ir._corpus)
    assert len(result) == len(answer)
    for posting in result:
        if posting not in answer:
            print("Missing Posting in the Posting List!")
    print("Leading Wildcard Query executed correctly!")

In [66]:
start = time.perf_counter()
answer = leadingWildcardQuery(ir, "*christ")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

[OC] Checking the receipts at the 99 cent store: did our suggested purchases actually turn out to be bargains?
Let's take a look at the 2020 NBA Free Agency class (it's pretty awful)
2019 NBA Free Agent Tracker
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
Lost in all the commotion is the fact that Raptors, reigning Champions, will now go another year without a Christmas game
Each teams current Under-25 Core
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
Every player drafted before 2016 that is still currently playing with the same team since their rookie season
At Some Point You Have to Feel Bad for Westbrook
2019 NBA Free Agent Tracker
[OC] How Every Team Could Trade For Russell Westbrook
What’s your ideal Christmas 2019 lineup?
National TV Schedule Predictions for Major Dates
When does the NBA 2019-2020 schedule get released?
2019-20 NBA Free Agents and Team Roster Tracker
Heres the Charlotte Hornets can win the 

In [67]:
verifyLeadingWildcard(ir, "christ", answer)

Leading Wildcard Query executed correctly!


And **General Wildcard Queries** too:

In [68]:
def verifyGeneralWildcard(ir:'IRSystem', wildcard:str, answer:list):
    wildcard += "$"
    i = wildcard.index("*")
    wildcard = wildcard[i+1:] + wildcard[:i]
    norm_wildcard = normalize(wildcard)
    posting_list = [term.posting_list for term in ir._index._dictionary if term.term.startswith(norm_wildcard)]
    posting_list = reduce(lambda x, y: x.union(y), posting_list)
    result = posting_list.getFromCorpus(ir._corpus)
    assert len(result) == len(answer)
    for posting in result:
        if posting not in answer:
            print("Missing Posting in the Posting List!")
    print("General Wildcard Query executed correctly!")

In [69]:
start = time.perf_counter()
answer = generalWildcardQuery(ir, "Dw*e")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

[OC] How My 3-Year-Old Says the Name of Every 2019 NBA All-Star
The last 9 All-Star Game MVPs have all played for California teams.
Each teams current Under-25 Core
Quick look at how every team has changed so far this offseason
2019 NBA Free Agent Tracker
A Russell Westbrook trade would be the big story, but Danilo Gallinari is still a damn good player too. What trade destinations make sense for him?
[OC] Top 100 Scorers in NBA Finals History in Terms of Points Per Game
2019-20 NBA Free Agents and Team Roster Tracker
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?
2019-20 NBA Free Agents and Team Roster Tracker
2019-20 NBA Free Agents and Team Roster Tracker
This will be the first season that no Finals MVP is playing for the team he won it with since the award’s inception
All-NBA teams based solely off of player headshots
I put NBA players into the Wu Tang name generator
Random OC: Which NBA Player's Full Name would produce the highest Scrabble Score?


In [70]:
verifyGeneralWildcard(ir, "dw*e", answer)

General Wildcard Query executed correctly!


Finally we can execute **Multiple Wildcard Queries**:

In [71]:
start = time.perf_counter()
answer = multipleWildcardQuery(ir, "L*b*n")
print(f"\nElapsed time was {(time.perf_counter()-start)*1000:.3}ms")

[Discussion] The NBA "superteam" era has temporarily halted for the "duo" era
Can we just appreciate Kawhi's decision as neutral fans?
2019-20 NBA Free Agents and Team Roster Tracker
A Lakers-Sixers final would be a hell of a frontcourt showdown
Chicago Bulls have quietly become obsessed with advanced analytics and had a really good low key offseason.
Some funny Chinese nicknames of NBA players and how these name came
In his age 34/35 season, Michael Jordan made $33,140,000; accounting for 123% of the salary cap
The Lakers have done a commendable job filling out their roster.
3 Reasons why Kevin Durant chose number 7.
National TV Schedule Predictions for Major Dates
The biggest red flag for each LA team.
It's the offseason. Here is my top 100 player list.
The Lakers should unironically start Caruso
If we are truly in the Era of "The Duo's" now, it's time to remake NBA Jam.
PSA: Not every Elite team is a "super team"
The Lakers have managed to dramatically improve the biggest weakness o